# Model Saving and Restore

- tf.train.Saver
tf 에서는 checkpoint를 모델로 주로 저장함

In [ ]:
import tensorflow as tf

In [ ]:
tf.reset_default_graph()
# random variable 생성
test = tf.Variable(tf.truncated_normal(shape=[3,3]),name="test")
test

## Saving
- meta : 그래프에 관한 정보
- 나머지 2개가 weigts에 대한 정보를 가지고 있음

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(test))
    save_path = saver.save(sess,"model/test_model.ckpt")
    print(save_path)

## Load
### 그래프는 잘 restore가 됬으나 저장할때와 값이 다름

In [ ]:
tf.reset_default_graph()
# meta 만 불러오는경우 그래프가 생성됨
saver = tf.train.import_meta_graph("model/test_model.ckpt.meta")
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run('test:0'))

### 읽어들인 후에 restore를 해줘야함 

In [ ]:
tf.reset_default_graph()
# meta 만 불러오는경우 그래프가 생성됨
saver = tf.train.import_meta_graph("model/test_model.ckpt.meta")
with tf.Session() as sess:
    # ckpt resotre를 하는 순간에 weigts 값을 가져오는 역할을 함 
    saver.restore(sess,"model/test_model.ckpt")
    print(sess.run('test:0'))

### 다른변수에 넣어보기  -> meta 없이
- transfer learning 시에 많이 사용하는 방식
- core 는 같은 name으로 사용해야 유리함

In [ ]:
tf.reset_default_graph()
# name 이 일치하는것으로 데이터를 가져옴 
x = tf.Variable(tf.truncated_normal(shape=[3,3]),name="test")
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess,"model/test_model.ckpt")
    print(sess.run("test:0"))

# Estimator

- tensorflow model 에 대응되는 기능
- serving을 위한 export하는 기능
- 코드의 재사용을 잘 하기위한 목적을 가지고 있음
- keras에서의 model 보다 estimator가 조금더 어려움 ㅠ
<br><br>
- keras의 장,단점
쉽다 <br>
디테일한 설정이 어려움 <br>
- tensorflow 의 장점
보다 자유로움  -> 즉, tf의 estimator 사용시 keras보다 디테일한 설정이 가능함<br>

## tf.estimator.Estimator 의 Class

### estimator의 action

- training
- evaluation
- prediction
- export for serving

### 입력 파라미터
- model_fn  
함수 <br>
features, lables, mode, params  <br>
입력 데이터에 대한 정보 <br>
- model_dir
모델 작업을 한 작업 후  저장할 디렉토리
- config
모델 설정 파일
- params 
모델 사용시 하이퍼 파라미터들을 정의

## Sample Model Function
- mnist 를 이용한 간단한 시나리오 작업

In [ ]:
import tensorflow as tf 

In [ ]:
def lenet():
    layers = tf.keras.layers

    model = tf.keras.Sequential([
        layers.Reshape(
            target_shape=[28, 28, 1],
            input_shape=(28 * 28,)),

        layers.Conv2D(
            filters=20,
            kernel_size=[5,5],
            padding='same',
            activation=tf.nn.relu),

        layers.MaxPooling2D(
            pool_size=[2,2]),

        layers.Conv2D(
            filters=50,
            kernel_size=[5,5],
            padding='same',
            activation=tf.nn.relu),

        layers.MaxPool2D(
            pool_size=[2,2]),

        layers.Flatten(),

        layers.Dense(
            units=500,
            activation=tf.nn.relu),

        layers.Dense(
            units=10),
    ])

    return model

### model function
- estimator spec 을 반환하는 함수 
- train dataset으로 부터 mini batch를 생성하는 역할을 함.
- 모델 fuction 하나가 train, eval, predict, serving 4가지 모두가 가능하도록 해야함

In [ ]:
def model_function(features, labels, mode, params):
    learning_rate = params.get('learning_rate',1e-3)
    # get the model
    model = lenet()
    
    # 4가지 기능을 모두다 하기 위해서 mode로 action을 구분 
    # 여기에서는 학습, 평가, 예측 3가지 기능 
    if mode == tf.estimator.ModeKeys.TRAIN:
        logits = model(features)

        loss = tf.losses.softmax_cross_entropy(
            onehot_labels=labels,
            logits=logits)

        accuracy = tf.metrics.accuracy(
            labels=labels,
            predictions=logits)

        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        learning_rate = tf.identity(learning_rate, name='learning_rate')

        train_op = optimizer.minimize(loss, global_step=tf.train.get_or_create_global_step())
        # loss 와 traionop을 가지고 train
        estimator_spec = tf.estimator.EstimatorSpec(
            mode=tf.estimator.ModeKeys.TRAIN,
            loss=loss,
            train_op=train_op)

    elif mode == tf.estimator.ModeKeys.EVAL:
        # train 할때의 그래프와 inference의 그래프는 다른 구조를 가짐
        # training = False 를 줘야함
        # ex) drop out 때문에 
        logits = model(features, training=False)

        loss = tf.losses.softmax_cross_entropy(
            onehot_labels=labels,
            logits=logits)

        accuracy = tf.metrics.accuracy(
            labels=tf.argmax(labels,axis=1),
            predictions=tf.argmax(logits,axis=1))
        # accuracy 를 eval_metrix_ops 로 넘겨주면 됨 
        estimator_spec = tf.estimator.EstimatorSpec(
            mode=tf.estimator.ModeKeys.EVAL,
            loss=loss,
            eval_metric_ops={
                'accuracy': accuracy
            })

    elif mode == tf.estimator.ModeKeys.PREDICT:
        logits = model(features, training=False)
        # label 도 넣지 않기때문에  데이터 입력이 없음 
        predictions = {
            'predictions': tf.argmax(logits,axis=1),
            'probabilities': tf.nn.softmax(logits)
        }
        
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predictions
        )

    return estimator_spec

# Sample Input Function
Train and test input functions which generates mini-batch samples from mnist dataset. <br>
keras의 fit generator 와 같은 느낌  <br>
dataset을 만드는경우에 아래코드와 같은 방식을 사용하는것이 좋음 <br>

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
DATADIR = 'data'
BATCH_SIZE = 128
mnist = input_data.read_data_sets(DATADIR, one_hot=True)
def mnist_gen(mode, batch_size): # yields (features, labels): range in [0,1], one_hot_labels
    mnist = input_data.read_data_sets(DATADIR, one_hot=True)
    if mode == 'train':
        while(True):
            yield mnist.train.next_batch(batch_size)
    
    if mode == 'test':
        for _ in range(10000//BATCH_SIZE):
            yield mnist.test.next_batch(batch_size)

def train_input_fn():
    tf_dataset = tf.data.Dataset.from_generator(lambda : mnist_gen('train',BATCH_SIZE), (tf.float32,tf.int32))
    return tf_dataset

def test_input_fn():
    tf_dataset = tf.data.Dataset.from_generator(lambda : mnist_gen('test',BATCH_SIZE), (tf.float32,tf.int32))
    return tf_dataset

###  Estimator의 생성
- train: Trains a model given training data input_fn. 
- evaluate: Evaluates the model given evaluation data input_fn.
- predict: Yields predictions for given features.
- export_saved_model: Exports inference graph as a SavedModel into the given dir.

전부다 함수 하나씩을 필요로 함! <br>

#### 입력 함수 출력해보기

In [ ]:
ds = train_input_fn()
ds

#### estimator 생성

In [ ]:
classifier = tf.estimator.Estimator(model_fn=model_function, model_dir='model_dir', params={'learning_rate':1e-3})
classifier

#### 생성한 estimator를 이용하여 학습, 평가 ,테스트 해보기
-> 사용시 cudnn 설정 한번더 확인하기 ! 

In [ ]:
classifier.train(train_input_fn, steps=2000)

In [ ]:
classifier.evaluate(test_input_fn)

In [ ]:
gen = classifier.predict(test_input_fn)
for _ in range(5):
    print(next(gen))